In [1]:
import scanpy as sc
import pandas as pd
import os
import h5py
import anndata as ad
from anndata.experimental import read_elem, sparse_dataset
import scipy.sparse
import numpy as np
import mudata as md
import snapatac2_scooby as sp

from scooby.utils.utils import read_backed
import h5py

## Test neurips

### Read in adata that is used to create the embedding

In [2]:
cell_type_key = 'tissue_label'

In [3]:
data_path = '/data/ceph/hdd/project/node_08/QNA/scborzoi/submission_data'

In [4]:
adata = sc.read(
    os.path.join(data_path, 'onek1k_training_data', 'OneK1K_only_immune.h5ad'), backed='r')

### Read in adatas that contain the profiles

In [5]:
sample = 'merged'

In [6]:
adata_plus = read_backed(h5py.File(os.path.join(data_path, 'onek1k_training_data', f'snapatac_{sample}_plus.h5ad')), "fragment_single") #sc.read(os.path.join(data_path, 'onek1k_training_data', f'snapatac_{sample}_plus.h5ad'))

In [7]:
adata_minus = read_backed(h5py.File(os.path.join(data_path, 'onek1k_training_data', f'snapatac_{sample}_minus.h5ad')), "fragment_single")

### Read in embedding

In [30]:
embedding = pd.read_parquet(os.path.join(data_path, 'onek1k_training_data', 'embedding_no_val_genes_fixed.pq'))

### Read in cell type to idx mapping

In [16]:
celltypes = pd.read_parquet(os.path.join(data_path, 'onek1k_training_data', 'celltype_fixed.pq'))

In [20]:
os.path.join(data_path, 'onek1k_training_data', 'celltype_fixed.pq')

'/data/ceph/hdd/project/node_08/QNA/scborzoi/submission_data/onek1k_training_data/celltype_fixed.pq'

### Write cell_type_index

In [10]:
cellindex = [
    {'celltype': cell_type, 
     'cellindex': list(np.where(adata_plus.obs.cell_label == cell_type)[0])} for cell_type in adata_plus.obs.cell_label.unique()
]

In [18]:
del cellindex

### Test that adata, and profile adata have the same order

In [21]:
assert (adata.obs_names == adata_plus.obs_names).all()

In [22]:
assert (adata.obs_names == adata_minus.obs_names).all()

### Test that embeddings have the correct order

In [23]:
assert (embedding.obs_names == adata_plus.obs_names).all()

## Test that celltypes index are correct

In [28]:
for i, row in celltypes.iterrows():
    print(row['celltype'])
    assert adata.obs[cell_type_key].iloc[row['cellindex']].unique() == row['celltype']

CD4+ TCM cell
NK cell
CD4+ T cell
CD8+ TEM cell
B cell
CD8+ TCM cell
CD4+ TEM cell
memory B cell
CD8+ T cell
CD56+ NK cell
CD16+ monocyte
MAIT cell
Treg memory
monocyte
gdT cell
CD4+ CTL cell
dnT cell
hematopoietic precursor cell
platelet
plasmablast
dendritic cell
plasmacytoid dendritic cell
